In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
from datetime import timedelta
from typing import List, Dict
import random

import pandas as pd
pd.set_option('display.max_rows', 200)
import numpy as np

from tqdm import tqdm

import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

from data_handlers.runtimes import *

In [22]:
experiment_path = pathlib.Path(f"/home/sterz/maci-docker-compose/maci_data/binary_files")
experiment_ids = [4] #[4, 5, 6, 7]

In [23]:
event_frame = parse_bundle_events(experiment_path, experiment_ids)

Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/503
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/497
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/509
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/531
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/546
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/567
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/510
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/528
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/522
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/555
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/558
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/552
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/525
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/489
Parsing /home/sterz/maci-docker-co

/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/

In [24]:
for bundle, group in event_frame[(event_frame["Event"] == "creation")].groupby("Bundle"):
    if len(group) != 1:
        print(bundle)
        print(group)

In [26]:
event_frame

,Simulation ID,Payload Size,Timestamp,Event,Node,Bundle,Software,Bundles per Second,CLA,# Nodes,# Payloads,Relative Timestamp
24520,487,1000,1900-01-01 07:34:08.979000,sending,n2,e291df62-984d-459a-9ddc-320f5e8419ba.file\n,Forban,1,default,32,1,0 days 00:00:00
24521,487,1000,1900-01-01 07:34:08.986000,reception,n2,e291df62-984d-459a-9ddc-320f5e8419ba.file\n,Forban,1,default,32,1,0 days 00:00:00.007000
24504,487,1000,1900-01-01 07:34:12.330000,sending,n3,e291df62-984d-459a-9ddc-320f5e8419ba.file\n,Forban,1,default,32,1,0 days 00:00:03.351000
24505,487,1000,1900-01-01 07:34:12.766000,reception,n3,e291df62-984d-459a-9ddc-320f5e8419ba.file\n,Forban,1,default,32,1,0 days 00:00:03.787000
24472,487,1000,1900-01-01 07:34:14.401000,sending,n4,e291df62-984d-459a-9ddc-320f5e8419ba.file\n,Forban,1,default,32,1,0 days 00:00:05.422000
...,...,...,...,...,...,...,...,...,...,...,...,...
3397,567,1000000,2023-02-12 11:26:17.992553,creation,n1,0cf0c3ec-d211-4a5d-b4fd-0652c23ebf9c.file,Forban,100,default,16,100,44967 days 00:59:14.448553
3398,567,1000000,2023-02-12 11:26:18.004821,creation,n1,8256f911-8aa6-4d0f-8c0a-cf3c853f2e47.file,Forban,100,default,16,100,44967 days 00:59:14.460821
3399,567,1000000,2023-02-12 11:26:18.016891,creation,n1,1344f3af-6ffd-4b72-8809-878439ba7ead.file,Forban,100,default,16,100,44967 days 00:59:14.472891
3400,567,1000000,2023-02-12 11:26:18.029015,creation,n1,a76f3d91-4c2b-4fa6-9e5e-df2e18d34d4e.file,Forban,100,default,16,100,44967 days 00:59:14.485015
